In [1]:
from tokenize import group

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy.dialects.mssql.information_schema import columns
import hashlib

plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine, column
import json
import datetime
from datetime import timedelta ,time, datetime
import re
import os
import sys
import gc
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean, Risk_Data
pd.set_option('display.max_columns', None)

In [2]:
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()
risk_data = Risk_Data()

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [4]:
sql = '''
select 
DISTINCT t1.order_number as 订单号, t1.create_time as 用户下单时间, tol.delivery_username 收货人名称
from db_digua_business.t_order t1 
left join db_digua_business.t_order_stages t2 on t1.id = t2.order_id 
left join db_digua_business.t_pay_record t3 on t2.id = t3.stage_id 
left join db_digua_business.t_order_logistics tol on tol.order_id=t1.id
where (t1.merchant_id = 23 || t1.activity_id = 68 || t1.remark like "%sgjh2024%" || t1.remark like "%sgjh2025%") 
and t1.`status` = 4 
and t2.`status` = 1 
and t2.sort = 2 
and t3.id is not null 
and t3.pay_type IN ("ZFBYWD", "ZFBYSQ")
and  DATE_FORMAT(t1.create_time, '%Y-%m-%d')>='2025-03-01'
-- and  DATE_FORMAT(t1.create_time, '%Y-%m-%d')<='2025-03-09';
'''


In [5]:

# with pd.ExcelWriter(f'F:/需求/线下小店/线下小店第二期扣款失败订单_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df.to_excel(writer, sheet_name='线下小店', index=False)
from apscheduler.schedulers.blocking import BlockingScheduler  
from apscheduler.schedulers.background import BackgroundScheduler 
# 每日的10点30分执行    
def my_job1():  
    today = datetime.today() 
    t_date = today.date().strftime('%Y%m%d%H')
    print(f"执行定时任务：现在是{t_date}的30分")
    path1 = r'\\digua\迪瓜租机\001线下小店/'    
    df = query(sql)
    df.to_excel(path1 + f'线下小店扣款失败数据_{t_date}.xlsx')
    del df
    gc.collect()
    print("回收内存执行完毕！\n")

scheduler1 = BackgroundScheduler()
scheduler1.add_job(my_job1, 'cron', hour=10, minute=25)   
# my_job1()
# 启动scheduler
scheduler1.start()

执行定时任务：现在是2025103100的30分
回收内存执行完毕！



In [6]:
# 每日的17点20分执行一次
def my_job2():  
    today = datetime.today() 
    t_date = str(datetime.now().strftime('%Y%m%d%H'))
    print(f"执行定时任务：现在是{t_date}的20分")
    path1 = r'\\digua\迪瓜租机\001线下小店/'    
    df = query(sql)
    df.to_excel(path1 + f'线下小店扣款失败数据_{t_date}.xlsx')
    del df
    gc.collect()
    print("回收内存执行完毕！\n")

scheduler2 = BackgroundScheduler()
# scheduler2.add_job(my_job2, 'cron', hour=17, minute=30)   
scheduler2.add_job(my_job2, 'cron', hour=17, minute=20) 
# 启动scheduler  
scheduler2.start()
# my_job2()

执行定时任务：现在是2025103017的20分
回收内存执行完毕！

